## Big Data Mining
### Project 2
### 2019-2020

Kyriakos   Zormpas  
Alexandros Zotos  
Theodora  Koutsothanasi  
Evangelia Santorinaiou  dsc19024  

## Libraries

In [ ]:
#!pip install namegenerator

In [12]:
import pandas as pd
import numpy as np

import namegenerator
from datetime import datetime, timedelta
import random
import time

### Dataset Creation

Our scenario in order to build our dataset is that we have all the restaurants of the world, name, id, when they were founded and their age, their coordinates, their speciality and their reviews. 

In [13]:
def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))

def random_date(start, end, prop):
    return str_time_prop(start, end, '%m/%d/%Y', prop)

To find the random latitude amd longitude we used worldcitiespop.csv and we kept the country and the city that match each combination. We did not use AccentCity column because it is the same with City but with accents thus it not provides additional information. Also we removed Population because most of the values were Nan and Region because we will not use it in our scenario.

In [14]:
def get_latitude_longititude():
    df = pd.read_csv('worldcitiespop.csv')
    return df.loc[:,['Country', 'City', 'Latitude', 'Longitude']]

In [18]:
df_lat = get_latitude_longititude()[:100]
df_lat

/home/eva/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,Country,City,Latitude,Longitude
0,ad,aixas,42.483333,1.466667
1,ad,aixirivali,42.466667,1.500000
2,ad,aixirivall,42.466667,1.500000
3,ad,aixirvall,42.466667,1.500000
4,ad,aixovall,42.466667,1.483333
5,ad,andorra,42.500000,1.516667
6,ad,andorra la vella,42.500000,1.516667
7,ad,andorra-vieille,42.500000,1.516667
8,ad,andorre,42.500000,1.516667
9,ad,andorre-la-vieille,42.500000,1.516667


In [19]:
restaurant_df = pd.DataFrame(columns=['id', 'name', 'cid', 'sid', 'rid', 'founded', 'age'])


for i in range(df_lat.shape[0]):
    r_date = random_date("01/01/2008", "12/31/2019", random.random())
    
    restaurant_df.loc[i] = [np.random.randint(100000, 999999), 
                                   namegenerator.gen(), 
                                   np.random.randint(100000, 999999), 
                                   np.random.randint(10000, 99999), 
                                   np.random.randint(100000, 999999),  
                                   r_date, 
                                   int(str(datetime.today())[:4]) - int(r_date[6:])]

In [20]:
restaurant_df

,id,name,cid,sid,rid,founded,age
0,463479,ready-khaki-chameleon,927000,83814,528662,05/29/2018,1
1,779995,blurry-amaranth-bandicoot,650460,29617,436098,02/20/2008,11
2,414349,skinny-cinnabar-bloodhound,677642,49969,687154,11/10/2016,3
3,150576,wheezy-chocolate-jellyfish,220914,83056,945190,08/07/2008,11
4,833286,lanky-tan-donkey,896076,39081,114218,12/17/2013,6
5,785251,cloudy-gamboge-shrimp,629071,27579,177727,01/26/2014,5
6,529901,leaky-gold-mastiff,524220,48084,374744,06/20/2009,10
7,791928,scummy-azure-echidna,532567,64770,993110,11/01/2017,2
8,657216,woolly-apricot-saola,865555,54461,751224,12/06/2011,8
9,358788,cloudy-jade-toad,426984,82267,950100,02/28/2009,10


In [21]:
coordinates = pd.DataFrame(columns = ['id', 'latitude', 'longitude', 'city', 'country'])

coordinates['id'] = restaurant_df['cid']
df_ll = get_latitude_longititude()[:100]
coordinates['latitude'] = df_ll['Latitude']
coordinates['longitude'] = df_ll['Longitude']
coordinates['city'] = df_ll['City']
coordinates['country'] = df_ll['Country']

coordinates

,id,latitude,longitude,city,country
0,852159,42.483333,1.466667,aixas,ad
1,539090,42.466667,1.500000,aixirivali,ad
2,228893,42.466667,1.500000,aixirivall,ad
3,778605,42.466667,1.500000,aixirvall,ad
4,648040,42.466667,1.483333,aixovall,ad
5,987159,42.500000,1.516667,andorra,ad
6,171746,42.500000,1.516667,andorra la vella,ad
7,656290,42.500000,1.516667,andorra-vieille,ad
8,703023,42.500000,1.516667,andorre,ad
9,741358,42.500000,1.516667,andorre-la-vieille,ad


In [28]:
speciality_df = pd.DataFrame(columns = [ 'id', 'name', 'calories', 'price', 'cuisine_name', 'is_vegan', 
                                        'is_gluten_free', 'is_lactose_free'])

specialityNamesList = [line.strip() for line in open("specialyNames.txt", 'r', encoding="utf8")]
cuisineNamesList = [line.strip() for line in open("cuisineNames.txt", 'r', encoding="utf8")]

speciality_df['id'] = restaurant_df['sid']
for i in range(restaurant_df.shape[0]):
    speciality_df.loc[i]['name'] = random.choice(specialityNamesList)
    speciality_df.loc[i]['calories'] = random.randrange(50, 1000)
    speciality_df.loc[i]['price']= random.randrange(1, 3500)
    speciality_df.loc[i]['cuisine_name'] = random.choice(cuisineNamesList)
    speciality_df.loc[i]['is_vegan'] = random.randrange(100) < 25
    speciality_df.loc[i]['is_gluten_free'] = random.randrange(100) < 15
    speciality_df.loc[i] ['is_lactose_free'] = random.randrange(100) < 15


In [29]:
speciality_df

,id,name,calories,price,cuisine_name,is_vegan,is_gluten_free,is_lactose_free
0,83814,"Vol-au-vent, with Oysters, a la Bechamelle",882,2124,Mediterranean cuisine,False,False,False
1,29617,"boiled potatoes, mustard sauce",292,1601,Laotian,False,False,False
2,49969,"A.C. Bourgogne RosÃ© ""Marsannay la CÃ´te"" 1976",75,3324,Mexican,False,False,False
3,83056,"Potatoes-- Sweet, Southern style",94,3232,Mangalorean,False,False,True
4,39081,Anglu rugtais,62,2804,Goan,False,False,False
5,27579,Scotch [Sour],212,3168,Balochi,False,True,True
6,48084,Vom Grill: Kalbssteak 150 g,356,627,Albanian,False,True,False
7,64770,"Fresh Asparagus, hollandaise sauce",773,300,Ukrainian,False,False,False
8,54461,"Drambuie, Cordial",404,1791,Lithuanian,False,False,False
9,82267,Chesire Cheese,434,1276,Crimean Tatar,False,False,False


In [37]:

review_df = pd.DataFrame(columns= ['id', 'name', 'rating', 'description'])

### Add Noise

One case we thought for noise is that some restaurants either do not have specility dish or their speciality is missing. Thus in restaurant dataframe in column sid we added some null values and also we changed some ids from speciality so that sid from restaurants and id from speciality do not much.

In [35]:
nn = int(restaurant_df.shape[0] * 1/100)

for i in range(nn):
    restaurant_df.loc[np.random.randint(0, restaurant_df.shape[0]-1)]['sid'] = 'null'
    speciality_df.loc[np.random.randint(0, restaurant_df.shape[0]-1)]['id'] = 
                      np.random.randint(1000, 9999)

In [36]:
restaurant_df

,id,name,cid,sid,rid,founded,age
0,131574,crappy-pear-vulture,152693,71811,808138,07/05/2015,4
1,330104,bumpy-vermilion-manatee,285009,88897,770318,02/13/2012,7
2,674697,seedy-puce-spider,678091,91070,978995,02/16/2011,8
3,882751,randy-fuchsia-havanese,198207,93764,857665,02/18/2014,5
4,952485,gummy-periwinkle-ibis,665064,75058,423828,01/11/2010,9
5,863697,greasy-viridian-binturong,893890,65207,664664,09/09/2009,10
6,811565,crabby-chestnut-penguin,240896,28884,264142,08/04/2015,4
7,853642,sleazy-burgundy-tiger,937418,75825,584804,05/09/2014,5
8,475729,thirsty-olivine-sheepdog,177631,77822,884088,06/26/2016,3
9,619160,thirsty-heliotrope-kiwi,535043,73542,727536,06/01/2015,4
